## Assignment 2: Hypothesis Testing

Hypothesis: Model trained on old data will do less well than more recent data.
Method: Taking responses from different years (e.g. 2018 vs 2020) to 4 intention questions, we train a logistic regression model and compare model performance on a held-out test set.

In [4]:
# HERE ARE ALL THE FUNCTIONS TO PREPROCESS DATA, TRAIN MODEL, GENERATE PREDICTIONS
# Note: since we use the PreFer training data, we cannot upload to github. If you wish to test the code to make sure it runs, you can use the fake data provided.

import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib
import random


def clean_df(df, background_df=None):
    """
    Preprocess the input dataframe to feed the model.

    Parameters:
    df (pd.DataFrame): The input dataframe containing the raw data (e.g., from PreFer_train_data.csv or PreFer_fake_data.csv).
    background (pd.DataFrame): Optional input dataframe containing background data (e.g., from PreFer_train_background_data.csv or PreFer_fake_background_data.csv).

    Returns:
    dfs (pd.DataFrame): The cleaned dataframes with only the necessary columns and processed variables per year.
    """

    # Grab variables depending on the year
    keepcols_2015 = [

    ]

    keepcols_2016 = [

    ]

    keepcols_2017 = [

    ]

    keepcols_2018 = [

    ]

    keepcols_2019 = [

    ]
    
    keepcols_2020 = [
        "nomem_encr",  # ID variable required for predictions,
        "birthyear_bg",
        "age_bg",
        "cf20m130", # Within how many years do you hope to have your first/next child?
        "cf20m128", # Do you think you will have children in the future?
        "cf20m129", # How many children do you think you'll have?
        "cf20m031" # What year did you marry?

    ] 

    # Keeping data with variables selected for each year
    dfs = {year: df[keepcols] for year, keepcols in zip(range(2015, 2021), [keepcols_2015, keepcols_2016, keepcols_2017, keepcols_2018, keepcols_2019, keepcols_2020])}

    # Calculate the median for each column
    medians = df.median()

    # Fill missing values with the median of each column for each year
    dfs = {year: df.fillna(medians) for year, df in dfs.items()}

    return dfs
    



def train_save_model(cleaned_df, outcome_df):
    """
    Trains a model using the cleaned dataframe and saves the model to a file.

    Parameters:
    cleaned_df (pd.DataFrame): The cleaned data from clean_df function to be used for training the model.
    outcome_df (pd.DataFrame): The data with the outcome variable (e.g., from PreFer_train_outcome.csv or PreFer_fake_outcome.csv).
    """
    
    ## This script contains a bare minimum working example
    
    # Combine cleaned_df and outcome_df
    model_df = pd.merge(cleaned_df, outcome_df, on="nomem_encr")

    # Filter cases for whom the outcome is not available
    model_df = model_df[~model_df['new_child'].isna()]  
    X = model_df.drop(['new_child', 'nomem_encr'], axis=1)
    y = model_df['new_child']

    # Define the model
    model = LogisticRegression(verbose=10)

    # Fit the model
    model.fit(X,y)

    # Save the model
    joblib.dump(model, "model.joblib")

    # Print progress
    print("Model saved.")



In [ ]:
def predict_outcomes(df, background_df=None, model_path="model.joblib"):
    """Generate predictions using the saved model and the input dataframe.

    The predict_outcomes function accepts a Pandas DataFrame as an argument
    and returns a new DataFrame with two columns: nomem_encr and
    prediction. The nomem_encr column in the new DataFrame replicates the
    corresponding column from the input DataFrame. The prediction
    column contains predictions for each corresponding nomem_encr. Each
    prediction is represented as a binary value: '0' indicates that the
    individual did not have a child during 2021-2023, while '1' implies that
    they did.

    Parameters:
    df (pd.DataFrame): The input dataframe for which predictions are to be made.
    background_df (pd.DataFrame): The background dataframe for which predictions are to be made.
    model_path (str): The path to the saved model file (which is the output of training.py).

    Returns:
    pd.DataFrame: A dataframe containing the identifiers and their corresponding predictions.
    """

    ## This script contains a bare minimum working example
    if "nomem_encr" not in df.columns:
        print("The identifier variable 'nomem_encr' should be in the dataset")

    # Load the model
    model = joblib.load(model_path)

    # Preprocess the fake / holdout data
    df = clean_df(df, background_df)

    # Exclude the variable nomem_encr if this variable is NOT in your model
    vars_without_id = df.columns[df.columns != 'nomem_encr']

    # Generate predictions from model, should be 0 (no child) or 1 (had child)
    predictions = model.predict(df[vars_without_id])

    # Output file should be DataFrame with two columns, nomem_encr and predictions
    df_predict = pd.DataFrame(
        {"nomem_encr": df["nomem_encr"], "prediction": predictions}
    )

    # Return only dataset with predictions and identifier
    return df_predict

In [ ]:
def score(prediction_path, ground_truth_path):
    """Score (evaluate) the predictions and write the metrics.

    This function takes the path to a CSV file containing predicted outcomes and the
    path to a CSV file containing the ground truth outcomes. It calculates the overall
    prediction accuracy, and precision, recall, and F1 score for having a child
    and writes these scores to a new output CSV file.

    This function should not be modified.
    """

    # Load predictions and ground truth into dataframes
    predictions_df = pd.read_csv(prediction_path)
    ground_truth_df = pd.read_csv(ground_truth_path)

    # Merge predictions and ground truth on the 'id' column
    merged_df = pd.merge(predictions_df, ground_truth_df, on="nomem_encr", how="right")

    # Calculate accuracy
    accuracy = len(merged_df[merged_df["prediction"] == merged_df["new_child"]]) / len(
        merged_df
    )

    # Calculate true positives, false positives, and false negatives
    true_positives = len(
        merged_df[(merged_df["prediction"] == 1) & (merged_df["new_child"] == 1)]
    )
    false_positives = len(
        merged_df[(merged_df["prediction"] == 1) & (merged_df["new_child"] == 0)]
    )
    false_negatives = len(
        merged_df[(merged_df["prediction"] == 0) & (merged_df["new_child"] == 1)]
    )

    # Calculate precision, recall, and F1 score
    try:
        precision = true_positives / (true_positives + false_positives)
    except ZeroDivisionError:
        precision = 0
    try:
        recall = true_positives / (true_positives + false_negatives)
    except ZeroDivisionError:
        recall = 0
    try:
        f1_score = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1_score = 0
    # Write metric output to a new CSV file
    metrics_df = pd.DataFrame(
        {
            "accuracy":     [accuracy],
            "precision":    [precision],
            "recall":       [recall],
            "f1_score":     [f1_score],
        }
    )
    return metrics_df


In [ ]:
# Load training data
df_train = pd.read_csv("training_data/PreFer_train_data.csv", low_memory=False)
df_train_outcome = pd.read_csv("training_data/PreFer_train_outcome.csv")

# Clean training data using clean_df from submission.py
df_train_cleaned = clean_df(df_train)

# Train model and save
train_save_model(df_train_cleaned, df_train_outcome)